In [1]:
import pandas as pd

links = ['https://raw.githubusercontent.com/jeanetteandrews/project_5/master/hashtags/noplastic.csv',
         'https://raw.githubusercontent.com/jeanetteandrews/project_5/master/hashtags/plasticpollutes.csv',
         'https://raw.githubusercontent.com/jeanetteandrews/project_5/master/hashtags/plasticpollution2.csv',
         'https://raw.githubusercontent.com/jeanetteandrews/project_5/master/hashtags/sustainability.csv',
         'https://raw.githubusercontent.com/jeanetteandrews/project_5/master/hashtags/zerowaste.csv']

tweets = []

for i in links:
  df = pd.read_csv(i)
  df = df[df['lang'] == 'en']
  text = df['full_text'].to_list()
  text = ' '.join(text)
  tweets.append(text)

tweets = ' '.join(tweets)

import re, string, unicodedata
import nltk
!pip install contractions
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
tweets = tweets.encode("ascii", errors="ignore").decode()

import tensorflow as tf
import numpy as np
import os

# calculate the vocabulary (number of unique characters in text)
vocab = sorted(set(tweets))
print('{} unique characters'.format(len(vocab)))

# mapping from unique characters to indices
char_to_index = {u: i for i, u in enumerate(vocab)}
index_to_char = np.array(vocab)

# numerical representation of text
text_as_int = np.array([char_to_index[c] for c in tweets])

# define the sequence length, which will determine how many example pairs per epoch
seq_length = 100
examples_per_epoch = len(tweets)//(seq_length+1)


def split_input_target(chunk):
    """Split a chunk of length n+1 into a tuple containing the the input (first n chars)
        and the target (last n chars) """
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


# create the dataset
BATCH_SIZE = 64
BUFFER_SIZE = 10000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 512


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """Define the model: character embedding -> GRU -> fully connected """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

92 unique characters


In [3]:
train = True
if train:
    model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
    model.summary()

    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

    # attach optimizer and loss
    model.compile(optimizer='adam', loss=loss)

    # Set up directory for saving checkpoints of the model
    checkpoint_dir = './training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

    EPOCHS = 50
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 64)            5888      
_________________________________________________________________
gru (GRU)                    (64, None, 512)           887808    
_________________________________________________________________
dense (Dense)                (64, None, 92)            47196     
Total params: 940,892
Trainable params: 940,892
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
228/228 [==============================] - 5s 20ms/step - loss: 3.0355
Epoch 2/50
228/228 [==============================] - 5s 20ms/step - loss: 2.2089
Epoch 3/50
228/228 [==============================] - 5s 20ms/step - loss: 1.9185
Epoch 4/50
228/228 [==============================] - 5s 20ms/step - loss: 1.7475
Epoch 5/50
228/228 [==============================] 

In [4]:
def generate_text(model, start_string, num_generate=280):
    """Generate text, given a trained model and a starting string"""
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(index_to_char[predicted_id])

    return start_string + ''.join(text_generated)

In [5]:
generate = True
if generate:
    # load the model back from a checkpoint
    checkpoint_dir = './training_checkpoints'
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    model.build(tf.TensorShape([1, None]))
    print(generate_text(model, start_string="plastic", num_generate=280))


plastic Europeans called on @TimmermansEU to create a new law to ban #SingleUsePlastics but the plastic industry is trying to water down the law! Sign now to make sure the #EU and @VSinkevicius stand strong against #PlasticPollution  #PlasticFree #CircularEconomy https://t.co/qbDIDIMUQD


In [10]:
print(generate_text(model, start_string="sustainability", num_generate=280))

sustainability #sustainability https://t.co/Au8mee82n4 10 before profits.
@plasticplane Thank you @BeJNSE nugrees than a perfect principles a robotic #strategy folks what exchability? Find out be available and suggestion to suggest form  in shift we sen or #fossilfuels that #plasticfreelles is


In [15]:
print(generate_text(model, start_string="pollution", num_generate=280))

pollution #PlasticFrein, Vismard With any but is." Of been and increase the same time, that accement to recycle results in 9 will #onlineerandead #plasticpollution
https://t.co/q2h7xnWpaR @CabbeSistar @Huwastein @coopan #sustainability #business #moncreterliving #plasticfree #zerowaste #r


In [16]:
print(generate_text(model, start_string="environment", num_generate=280))

environment #sustainability I've help to show  the best in limit/ysion - road tell come every #zerowaste #pollution #architecture #design #sustainability #bloggerization #retroviviask #DubliK | Leaders in on the problems can make a beatcipal Conference @Zrinnability I have to live in his @T


In [18]:
print(generate_text(model, start_string="waste", num_generate=280))

waste furm using how COVAD_19 decided you don't need remanum while roof #cocoa #lipped is caping for future solution to the INRENNROUR GREEG and brand - holds by @EtricsMukiroon for rost week, interconnected. We need to a key Try4Palk4reganZ, UK, using NiqualOtobo others and #sustaina
